# Data preparation for the Energy-based diffusion model

Compared to the similarity-kernel based generation, the energy-based diffusion model is specifically trained to generate molecules. Thus, we have to ensure a proper training/test split. To make this more challenging we will do a scaffold split and stratify the split by the number of heavy atoms.

In [1]:
import ase
import ase.io as aio
from rdkit import Chem

In [3]:
all_qm9 = aio.read("/home/rokas/Programming/data/qm9_full_data_all_props.xyz", index=":", format="xyz")

KeyboardInterrupt: 